In [ ]:
import os
import torch
import numpy as np
import xarray as xr
import netCDF4 as nc
import pandas as pd
import simpy as sp
import logging
import multiprocessing as mp
from multiprocessing import Pool

In [3]:
import cdsapi

In [4]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [5]:
os.chdir('..')

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from datasets.net_cfd import load_netcdf
from datasets.utils import unzip_csv_or_nc_batch

# Koopman Optimization of incomming Cargo Ships' Arrangement w.r.t. energy consumption

## Koopman Analysis

When looking into stadard Fourier Analysis, we describe a linear system between the actual oscillators and the desired output space, minimizing the squared error:

$\begin{equation}E\left(K, \omega\right) = \sum_{t=0}^{T-1} \left(x - K \Omega(\omega t) \right)^2\end{equation}$

With $E$ being the actual error function between our ground truth $x$ and the oscillator $\Omega(\omega', t)$.


The oscillator $\Omega(\omega t)$ is defined in the same manner as in the Fourier wavelets:

$
\begin{equation}
\Omega(\omega t) = \begin{pmatrix} 
    \sin(\omega_1 t) \\
    \vdots \\
    \sin(\omega_N t) \\
    \cos(\omega_1 t) \\
    \vdots \\
    \cos(\omega_N t)
\end{pmatrix}
\end{equation}
$

In Koopman Analysis, however, we look into any type of function $f_{\Theta}$, mostly non-linear or at least quasi-linear:

$\begin{equation}E\left(f_{\Theta'}(\Omega(\omega', t)) | x, \Theta', \omega'\right) = \sum_{t=0}^{T-1} \left(x - f_{\Theta}(\Omega(\omega' t), \Theta') \right)^2\end{equation}$

Furthermore, from this error function, we can derive some type of pseudo $log$-likelihood, using any kind of error function for oscillators and periodic frequency elements:

$\begin{equation}\log L\left(\Theta', \omega'\right) = - E\left(f_{\Theta'}(\Omega(\omega' t)) | x, \Theta', \omega'\right) \end{equation}$

In this case we optimize our non-liearity $\Theta' \rightarrow \Theta$ and our frequencies $\omega' \rightarrow \omega$.

Here for such a pseudo-likelihood, we would use a softmax function over all samples $n$ and for all target dimensions $d$ to guarantee a distribution:

$\begin{equation}L\left(\Theta', \omega'\right) = \frac{\exp\left(\log L_{n, d}\left(\Theta', \omega'\right)\right)}
{\sum_{n=0}^{N-1} \sum_{\forall d} \exp\left(\log L_{n, d}\left(\Theta, \omega'\right)\right)} \end{equation}$




## ERA5 Around Hamburg Harbor

ERA5 is a dataset which provides hourly maritime weather data.<br>
Its API does only provide single point measurements, but we can play a little and grab this data for multiple points all together.

![Map with coordinated of HH harbor.](../img/map_small_hhh.png)<br>
*Coastal map. The red point is at coordinates lon 10 and lat 53.5, around Hamburg Harbor.*<br>
*The blue dot is representing the initial radius taken when data mining the region of interest.*<br>
*One can see that that the mean on that point needs to be far more at the caostal line to get sufficiant forecasting!*

In [8]:
# params
dataset = "reanalysis-era5-single-levels-timeseries"
center_lon, center_lat = 7.25, 53.5
radius = 2.75
step = 0.25
start_date = '2025-03-01'
end_date = '2025-09-01'
output_dir = f"./data/{start_date}--{end_date}"
os.makedirs(output_dir, exist_ok=True)

import glob

for fp1 in glob.glob('./data/*/*.nc'):
    fp1 = os.path.abspath(fp1)
    fp1 = fp1.replace('\\', '/')
    fp0 = '/'.join(fp1.split('/')[:-1])
    fn1 = fp1.replace(fp0, '')
    fn2 = fn1.replace('.nc', '_compr.zip')
    fp2 = fp0 + fn2
    os.rename(fp1, fp2)
    

In [9]:
lon_vals = np.arange(center_lon - radius, center_lon + radius + step, step)
lat_vals = np.arange(center_lat - radius, center_lat + radius + step, step)

# Create list of (lon, lat) pairs
tasks = [(lon, lat) for lon in lon_vals for lat in lat_vals]

In [ ]:
def data_filename(lon, lat, output_dir=output_dir) -> str:
    return os.path.join(output_dir, f"data_lon{lon:.2f}_lat{lat:.2f}_compr.zip")

def data_already_exists(lon, lat) -> bool:
    return os.path.isfile(data_filename(lon, lat))

def download_data(lon, lat, start_date=start_date, end_date=end_date) -> None:
    try:
        request = {
            "variable": [
                "2m_dewpoint_temperature",
                "mean_sea_level_pressure",
                "skin_temperature",
                "surface_pressure",
                "surface_solar_radiation_downwards",
                "sea_surface_temperature",
                "surface_thermal_radiation_downwards",
                "2m_temperature",
                "total_precipitation",
                "10m_u_component_of_wind",
                "10m_v_component_of_wind",
                "100m_u_component_of_wind",
                "100m_v_component_of_wind",
                "mean_wave_direction",
                "mean_wave_period",
                "significant_height_of_combined_wind_waves_and_swell"
            ],
            "location": {"longitude": lon, "latitude": lat},
            "date": [f"{start_date}/{end_date}"],
            "data_format": "csv"
        }
        fname = data_filename(lon, lat)
        client = cdsapi.Client()
        client.retrieve(dataset, request).download(fname)
    except Exception as e:
        logging.error(f'Cannot download file {fname}: \n{e}')

def retrieve_point(args) -> None:
    lon, lat = args
    if not data_already_exists(lon, lat):
        download_data(lon, lat)

for tsk in tasks:
    retrieve_point(tsk)

2025-10-18 02:11:28,804 WARNING [2025-03-17T00:00:00] Please be aware that the generation of this dataset is using an alternative source for the ERA5 data and may be subject to changes over time (e.g. file format, data file structure, deprecation etc). This dataset should therefore be regarded as “experimental” and is **not recommended for use in a production environment**. 

Notification of changes via this catalogue entry banner and/or in the [Forum](https://forum.ecmwf.int/) will be provided on best efforts.
2025-10-18 02:11:28,806 INFO Request ID is f86fadf4-6e9c-46fc-8a8a-99bb49cae700
2025-10-18 02:11:28,862 INFO status has been updated to accepted
2025-10-18 02:11:42,377 INFO status has been updated to running
2025-10-18 02:11:50,036 INFO status has been updated to accepted
2025-10-18 02:12:01,488 INFO status has been updated to successful
2025-10-18 02:12:03,538 WARNING [2025-03-17T00:00:00] Please be aware that the generation of this dataset is using an alternative source for t

ValueError: No objects to concatenate

In [9]:
unzip_csv_or_nc_batch(output_dir)

In [ ]:
dfs = []
for lon in lon_vals:
    for lat in lat_vals:
        fname = os.path.join(output_dir, f"data_lon{lon:.2f}_lat{lat:.2f}.csv")
        if os.path.isfile(fname):
            df_ = pd.read_csv(fname)
            dfs.append(df_)

raw_df = pd.concat(dfs, axis=0, ignore_index=True)
raw_df = raw_df.rename(columns={
    'valid_time': 'time',
    'latitude': 'lat',
    'longitude': 'lon'
})

In [11]:
raw_df.shape

(2348760, 8)

In [12]:
raw_df[~raw_df['mwd'].isna()]

,time,mwd,mwp,swh,latitude,longitude,lon,lat
35520,2025-03-01 00:00:00,351.44620,4.387406,0.538020,53.0,4.5,4.5,52.75
35521,2025-03-01 01:00:00,351.37990,4.371970,0.530165,53.0,4.5,4.5,52.75
35522,2025-03-01 02:00:00,351.43652,4.339733,0.525052,53.0,4.5,4.5,52.75
35523,2025-03-01 03:00:00,351.70572,4.303671,0.521995,53.0,4.5,4.5,52.75
35524,2025-03-01 04:00:00,351.96213,4.307576,0.513879,53.0,4.5,4.5,52.75
...,...,...,...,...,...,...,...,...
2326555,2025-09-01 19:00:00,168.64174,2.854231,0.189028,55.0,10.0,10.0,55.00
2326556,2025-09-01 20:00:00,169.11900,2.827129,0.168265,55.0,10.0,10.0,55.00
2326557,2025-09-01 21:00:00,169.56084,2.795815,0.150129,55.0,10.0,10.0,55.00
2326558,2025-09-01 22:00:00,169.36804,3.035513,0.105434,55.0,10.0,10.0,55.00


In [ ]:
raw_df['miss'] = raw_df['mwd'].isna()
sum_df = raw_df.groupby(['lon', 'lat'])[]

: 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6), subplot_kw={"projection": ccrs.PlateCarree()})

# Add fine coastlines and borders
ax.add_feature(cfeature.COASTLINE.with_scale("50m"), linewidth=0.5)
ax.add_feature(cfeature.BORDERS.with_scale("50m"), linewidth=0.3)
ax.add_feature(cfeature.LAND, facecolor="lightgray", alpha=0.6)
ax.add_feature(cfeature.OCEAN, facecolor="whitesmoke", alpha=0.7)

# Compute zoom area with margin
margin = 2
ax.set_extent([
    raw_df["lon"].min() - margin,
    raw_df["lon"].max() + margin,
    raw_df["lat"].min() - margin,
    raw_df["lat"].max() + margin
], crs=ccrs.PlateCarree())

# Plot points (red = missing, blue = present)
colors = raw_df["miss"].map({True: "red", False: "blue"})
ax.scatter(raw_df["lon"], y=raw_df["lat"], c=colors, s=40, edgecolor="black", linewidth=0.3, transform=ccrs.PlateCarree())

plt.title("Missing (red) vs Non-missing (blue)", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Convert time column to datetime if not already
raw_df['time'] = pd.to_datetime(raw_df['time'])
raw_df['hour'] = raw_df['time'].dt.to_period('H')

# Aggregate to monthly mean (modify 'value_variable' to actual variable name)
variables = [col for col in raw_df.columns if col not in ['time', 'month', 'longitude', 'latitude']]
monthly_data = raw_df.groupby(['hour', 'longitude', 'latitude'])[variables].mean().reset_index()

# Save to parquet
monthly_data.to_parquet("hourly_era5_data.parquet")


KeyError: 'time'

## HAVOC